# The predictive distribution

Under the bayesian point of view of a regression of linear basis functions, we are interested in making predictions of $t$ given an unseed value $\bf x$.

Consider
$$
    p(t | {\bf t}, {\bf x}, \alpha, \beta) = \mathcal{N}({\bf w} | m_N, S_N) \mathcal{N}(t | {\bf w}^T {\bf \phi(x)}, \beta^{-1})
$$



In [1]:
import numpy as np
import matplotlib.pyplot as plt